# `aws dynamodb`

the previous two lecture topics were about the difference between relational databases and `nosql` databases, and an overview of one particular relational database (`postgres`) which can be implemented using the `aws rds` service.

now, let's turn our attention to `nosql` options

`aws` has one natively managed `nosql` service: `dynamodb`.

(with a name like `dynamo`, it must be *VERY* cool)

`dynamodb` is both a key-value and a document store. it is primarily designed for very low latency applications (large-scale web projects, phone apps, *etc.*).

like other `nosql` databases, it doesn't propose or mandate any schema, which is kind of the point: any file can come in at any time with arbitrary contents, and we don't want it to break our super l33t mobile game.

in all honesty, while you may come across these databases in the data science wilderness, it is (in my experience, at least) *much* less common than relational databases.

data scientists are usually working with structured data, so imposing a structure (i.e. a relation) makes sense. the use case for document stores generally has much more to do with software and data engineering than analysis.

all that being said, it is still good to know about them -- you don't always get to determine what structure your data comes in, and if you're provided data in a `nosql` database such as this one (or, more commonly, `mongodb`), it'd be pretty great if *you* knew how to handle it.

so let's figure that out!

## how is data arranged in `dynamodb`?

hat tip to [the docs](http://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Introduction.html) which we are following in our intro discussion

### tables, items, and attributes

the nomenclature and conceptual arrangement of items in `dynamodb` is not very different from that in a regular ol' relational database. it has:

+ **tables**: a high-level conceptual distinction (e.g. `Person` and `Company`)
+ **items**: similar to records, these are individual instances of the concept described in the **table** (e.g. a "Zach Lamberty" item)
+ **attributes**: similar to a single field in a record, these are units of information (under *no* schema) about one of the above **items** (e.g. `age`, `address`). these can be hierarchical

there are relationships between those three levels:

+ **tables** are collections of **items**
+ **items** are collections of **attributes**

+ **primary keys**: every **item** must be uniquely identified by one or two of its attributes
    + looking up a record by its **primary key** is *instantaneous* in `dynaomdb`. this is equivalent to a `python` dictionary lookup (the **primary key** is converted into a hash key)
    + a single element **primary key** determines where a record is stored
    + a second element in a **primary key** determines the order of records in that storage space

### an example

take the following records as a single `Person` table:

```json
[
    {
        "PersonID": "chb49",
        "LastName": "Blancarte",
        "FirstName": "Carlos"
    },
    {
        "PersonID": "rzl5",
        "LastName": "Lamberty",
        "FirstName": "Ron",
        "MiddleName": "Zach",
        "Title": "Adjunct"
    }
    {
        "PersonID": "jks49",
        "LastName": "Shaw",
        "FirstName": "J",
        "MiddleName": "Ken",
        "Title": "Director of Graduate Studies"
    }
]
```

in the above, the `PersonID` serves as a unique identifier **attribute** of each **item**. also, note that the schema is not consistent -- it doesn't have to be.

### indexes

in analogy with relational databases, `dynamodb` also supports *indexes* on this data. you can think of indexes as effectively pre-caching the query results based on certain fields' values.

in the above `Person` example, querying for records by the primary key (`PersonID`) is instantaneous. we can also create an *index* on `LastName` to make lookup based on last name much faster.

faster is inherently better and no matter what we should always care that things are fast

just kidding.

why does this actually matter? often times it won't. but you should know that when interacting with a `nosql` database (or a relational database, for that matter):

*when doing queries, search on primary keys or indices will be fast(er), and search on other fields or columns will be slow(er). if query speed is a bottleneck in some process you are working on, consider discussing the possibility of creating an index with your dba*

### streams

a particularly useful feature of `dynamodb` (compared to other `nosql` databases) is **streams**, which are effectively `aws event` notifications (such as those we used to trigger `lambda` functions, hint hint hint) about changes to the database.

"changes" in this context refers to changes to **items** in **tables**:

+ a new item is inserted
+ an existing item is updated
+ an existing item is removed

## how is data added to `dynamodb`?

for most relational databases (and `postgres` in particular) we make a connection to the database with some program (e.g. `psql`) and execute `sql` statements within that session. 

under the hood, the relational database developers have created their own low-level protocol for allowing those connections and submitting those statements, and our `psql` program is really just sending messages in that protocol over the wire to the database.

`dynamodb`, by comparison, is an inherently web-centric database, which means that *all* of the requests that are sent to and from the database are exclusively `http` messages -- *including* things like table creation, item insert or update, and item deletion.

for a general service, `aws` will expose a `REST api`, will implement this in `boto3`, and will expose the most common actions via the `aws cli`.

for `rds`, this meant we could create database *instances*, or inquire about their state (up and running, backing up, etc), but if we wanted to perform any database options we had to use the database tools.

for `dynamodb`, there is no such separation between admin tools and database tools

this is all to say: records are added via `http` messages, related `boto3` statements, or `aws cli` commands

sidenote: [here is a really great presentation](https://wulczer.org/postgres-on-the-wire.pdf) about the `postgres` protocol. messages are transferred from client to server using TCP (just like `http` messages), but the internal structure is *very* different

### an example: creating a table and inserting a record

the creation of a `dynamodb` is done via the [`CreateTable`](https://docs.aws.amazon.com/amazondynamodb/latest/APIReference/API_CreateTable.html) endpoint, or [the `boto3 dynamodb resource create_table` function](http://boto3.readthedocs.io/en/latest/reference/services/dynamodb.html#DynamoDB.ServiceResource.create_table).

as with other `aws` services, all of this can be done via [the `dynamodb` web console](https://console.aws.amazon.com/dynamodb/home)

the table creation command is *asynchronous*, which means that we will receive an *immediate* response to our request to create a table even though that request won't be *completed* by that point

in order to add records to that we will need to poll the state of the database (with the `describe_table` function) before attempting to insert a new item. fortunately, `boto3` has taken care of that for us

<div align="center">**mini exercise: create a simple `dynamodb` table and poll for creation**</div>

run the following `python` code to create a table `Person` and insert some items:

In [ ]:
import boto3, time

session = boto3.session.Session(profile_name='gu511')
dynamodb = session.resource('dynamodb')

# create the table
person = dynamodb.create_table(
    TableName="Person",
    AttributeDefinitions=[
        {'AttributeName': 'guid', 'AttributeType': 'S'}
    ],
    KeySchema=[
        {'AttributeName': 'guid', 'KeyType': 'HASH'}
    ],
    ProvisionedThroughput={'ReadCapacityUnits': 1, 'WriteCapacityUnits': 1},
)
print('immediate status: {}'.format(person.table_status))

# wait until it's up
person.wait_until_exists()
print('table exists!')

we have successfully create a table using the `boto3` library! let's add some records to it.

we could add any 1 record with the `person.put_item` method, but we could also create a batch writer (using a context manager) to take many items and write them in larger groups ("batches") of 25 or fewer records.

In [ ]:
# it's up! insert some record
items = [
    {
        "guid": "chb49",
        "LastName": "Blancarte",
        "FirstName": "Carloa",
        "Title": "TA"
    },
    {
        "guid": "rzl5",
        "LastName": "Lamberty",
        "FirstName": "Ron",
        "MiddleName": "Zach",
        "Title": "Adjunct"
    },
    {
        "guid": "jks49",
        "LastName": "Shaw",
        "FirstName": "J",
        "MiddleName": "Ken",
        "Title": "Director of Graduate Studies",
        "Office": "327",
    }
]
with person.batch_writer() as batch:
    for item in items:
        batch.put_item(Item=item)

go check out those records in [the `dynamodb` web console](https://console.aws.amazon.com/dynamodb/home?region=us-east-1#tables:selected=Person).

### updating records

once inserted, items are updated using the `UpdateItem` endpoint, which is availabe in `boto3` as a `DynamoDB.Table.update_item` method. we can use that to fix the annoying typo in Carlos' first name:

In [ ]:
person.update_item(
    Key={'guid': 'chb49'},
    UpdateExpression="SET FirstName = :fn",
    ExpressionAttributeValues={':fn': 'Carlos'}
)

of course, you can also manually edit any record via [the web console]()https://console.aws.amazon.com/dynamodb/home?region=us-east-1#tables:selected=Person

## how do I get data out of `dynamodb`?

as I said previously, for most traditional data science applications where *you* control data acquisition and persistence, it will generally not make sense to put data into a key-value or document store like `dynamodb`.

that being said, most of the data in the world wasn't persisted to a database *by* and *for* you, so other people made other decisions with other things in mind. maybe that awesome clickstream data you want to model just so happened to fit the use case of `dynamodb`, and the web developers went with what made sense.

in other words: it's nice to know how to write and administer `dynamodb`, but it's better to know how to get data from it.

there are three basic ways to get data (listed here by the `REST api` endpoint names):

1. `GetItem`: pull a single item by its primary key
2. `Query`: pull the set of items which all share a primary key, possibly with a condition on the sort key
3. `Scan`: pull *all* items

the first two (`GetItem` and `Query`) are for fast access (i.e. the query knows the physical location of the record "in the cloud"), and they involve the primary key. `GetItem` uses an exact primary key, and `Query` requires a composite primary key (two values, not just one).

the last (`Scan`) is a slow, full-table query

### `GetItem`

the primary key (either a single partition key or a partition key with a sort key) is a unique identifier of an item in `dynamodb`. we can, of course, pull up that record by that key:

In [ ]:
person = dynamodb.Table('Person')
response = person.get_item(Key={'guid': 'rzl5'})
response['Item']

this method is equivalent to the `sql` statement

```sql
select * from Person where guid = "rzl5"
```

often in `sql` statements such as the above we don't want to return every value -- we only want the first and last name, say.

```sql
select FirstName, LastName from Person where guid = "rzl5"
```

we can also limit the fields returned by this request with the `ProjectionExpression` parameter (a list of the fields we want returned)

In [ ]:
response = person.get_item(Key={'guid': 'rzl5'}, ProjectionExpression='FirstName,LastName')
response['Item']

### `Query`

for a table with only one element in its primary key (such as our `Person` example), the only meaningful way to access records with that key is via the `GetItem` endpoint.

for tables with compound primary keys (a hash key and a sort key), there is a possible generalization called a `Query`

suppose we didn't have a `guid` as a primary key in `Person`, but instead we had a primary key that was `(LastName, FirstName)` (and let's ignore the complication that this is definitely not a good idea since first and last name pairs are far from unique)

our records would then be

```json
[
    {
        "LastName": "Blancarte",
        "FirstName": "Carlos"
    },
    {
        "LastName": "Lamberty",
        "FirstName": "Ron",
        "MiddleName": "Zach",
        "Title": "Adjunct"
    },
    {
        "LastName": "Shaw",
        "FirstName": "J",
        "MiddleName": "Ken",
        "Title": "Director of Graduate Studies"
    }
]
```

also, now, suppose I had been entered incorrectly at first as just "Zach Lamberty". there might then be two records for me:

```json
[
    {
        "LastName": "Lamberty",
        "FirstName": "Ron",
        "MiddleName": "Zach",
        "Title": "Adjunct"
    },
    {
        "LastName": "Lamberty",
        "FirstName": "Zach",
        "Title": "None"
    }
]
```

we could use the previous `GetItem` record to find either *one* of these records, but we could use `Query` to find *both* of them. this requires what `aws` calls a `KeyConditionExpression`, a boolean query string *a la* the `WHERE` clauses in standard `sql`:

```python
response = person.query(
    KeyConditionExpression='LastName = :ln',
    ExpressionAttributeValues={':ln': 'Lamberty'}
)
response['Item']
```

the `KeyConditionExpression` abstraction allows for arbitrarily complex expressions to be described in the same way, but also assists (on the `dynamodb` side, not the developer side) in query validation.

finally, it is possible to perform more advanced logic on other attributes using a `FilterExpression`. for example, finding all `Lamberty` records where the Title is `Adjunct`:

```python
response = person.query(
    KeyConditionExpression='LastName = :ln',
    FilterExpression='Title = :pos'
    ExpressionAttributeValues={
        ':ln': 'Lamberty',
        ':pos': 'Adjunct'
    }
)
response['Item']
```

one important note: this is all ***still*** isolated to items that have a shared primary key value. In the `Person` example, both `GetItem` and `Query` will be isolated to within one `guid`. in the modified `Person` example, they will both necessary be isolated to within one `Lastname`.

to pull records that have *different* hash key values, you have to use the slower `Scan` endpoint

### `Scan`

sometimes your interest isn't limited to the specific fields in the primary key. for example, suppose I was interested in all of the items in `People` with a `Title` of `TA`. this specifically cuts across primary keys, so `GetItem` and `Query` are pointless to me.

the `Scan` endpoint reads every record in the database and returns them (possibly filtered down based on a filter expression). this is analogous to the `sql` statement

```sql
select * from People
/* optionally
where some_condition = True
*/
```

the `Scan` statement is striaghtforward

In [ ]:
response = person.scan()
response['Items']

`Scan` operations are much more expensive (both in terms of resources and dollars) than `GetItem` or `Query` expressions. `aws` strongly recommends you avoid them, but it is sometimes unavoidable -- especially when the database was created for some *other* purpose (e.g. easy website traffic monitoring or construction keyed on specific users) and is being analyzed now to look at a property along a different dimension (e.g. traffic spikes as a function of time)

let's scan a particularly special table (one I've had up and running for a few days now)

In [ ]:
airportweather = dynamodb.Table('AirportWeather')
reports = airportweather.scan()
print(reports['Count'])
reports['Items'][0]

and look what `pandas` can do

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize

df = json_normalize(reports['Items'])
df.head()

### indexes

at this point you would be right to be thinking these options are pretty limited. basically, you can *quickly* pull up a record or several records which all have one shared value for *an important* key, or you can slowly pull up all records which have a shared value or are in a preferred range for *the other* keys.

**indexes** are a way of elevating "other" keys to "important" keys and open up faster query options without changing the fundamental structure (the primary key constraint) of the database

under the hood, indexes are basically just lookup tables. for each record in an existing table (where the primary key is the important thing), we add a record to the index table where the *index keys* are the new primary keys and the attributes are that base table record's primary key.

#### creating an index

creating an index on `Title` through [the web interface](https://console.aws.amazon.com/dynamodb/home?region=us-east-1#tables:selected=Person) is pretty straight-forward, so after we've done that let's also create one using `boto3`

In [ ]:
person.update(
    AttributeDefinitions=[
        {'AttributeName': 'LastName', 'AttributeType': 'S'},
        {'AttributeName': 'FirstName', 'AttributeType': 'S'}
    ],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': 'LastName-FirstName-index',
                'KeySchema': [
                    {'AttributeName': 'LastName', 'KeyType': 'HASH'},
                    {'AttributeName': 'FirstName', 'KeyType': 'RANGE'}
                ],
                'Projection': {'ProjectionType': 'ALL'},
                'ProvisionedThroughput': {
                    'ReadCapacityUnits': 1,
                    'WriteCapacityUnits': 1
                }
            }
        }
    ]
)

note: `boto3` doesn't provide a nifty polling function (like `wait_until_exists`), so we have to implement our own. We can do this as follows:

In [ ]:
import time

person.load()
waitseconds = 5
while person.table_status != 'ACTIVE':
    time.sleep(waitseconds)
    person.load()

#### using an index

the point of creating an index is to allow for fast searches (*i.e.* `Query` -- `GetItem` only applies to table primary keys) on non-primary-key attributes. Let's test that out by selecting all records with a `Title = "TA"`:

In [ ]:
person.query(
    IndexName='Title-index',
    # we can include index elements now:
    KeyConditionExpression='Title = :t',
    ExpressionAttributeValues={':t': 'TA'}
)

*note*: indexes sound awesome and they can be pretty helpful, so you might ask: why not just create an index for every column? the answer is that they are expensive, both in terms of resources and dollars, but also in terms of performance (every new record you write to a table updates every index on that table as well)

#### indexes and dbas in the real world 

there is a very particular set of circumstances that would make this index discussion relevant to you and a project on which you might work:

+ your source data is `dynamodb`
    + generally, though, the following also applies to the analogous concepts in `rdbm`s
+ the existing primary keys and indices do not cover the most important query keys of your modelling approach
    + *e.g.* data is keyed on user and sorted on time and you are interested in `ip` address (no index)
+ your queries and ingestion process is unacceptably slow
    + this is often overlooked: just because your process is slower than it could be, that doesn't mean it's too slow!
    + it has to be unacceptable to your client / project, not just to you ;)

if your scenario matches these criteria, it's time to work with the `dba` to add an index. depending on the size of the data, this could be a pretty sizeable ask, so be prepared for some pushback

## authentication

finally, `dynamodb` has a slightly different authentication method than `rds`.

`rds`, if you recall, was effectively just `rdbm`s installed on `aws`-managed `ec2` instances. authentication and access to those databases was managed in two ways

1. network level: controlling connections to that `ec2` instance
2. database level: creating users on the database and granting them permissions

`dynamodb`, being a fully integrated service, controls permissions in the same way as other `aws` services: via `iam`. there are a few basic ways to give a user or program access to your `dynamodb` tables:

+ `iam` users
    + create an `iam` user and assign that user specific permissions, then use access keys for programmatic access
    + this is what I did for you all to access my [`AirportWeather` table](https://console.aws.amazon.com/dynamodb/home?region=us-east-1#tables:selected=AirportWeather)
+ `iam` roles
    + federated user access
    + `aws` service access
    + `ec2` application access

note: to clean up after yourself here, simply run:

In [ ]:
person.delete()
person.wait_until_not_exists()
print('"Person" table has been deleted')

note: if you are looking for further information, you should check out the [`dynamodb` and `python` "getting started" documentation](http://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.01.html)

advanced topics

+ eventual consistency
    + if you have competing writes, there is no transactional saving element
    + they suggest you use *conditional writes*, comparing updated values to previous values and rejecting writes if they are not consistent with *a* previous read
    + these seems like a total dealbreaker in shared systems. this feels like a very bad idea for multi-user update environments...

<!--div align="center">***DROP joke WHERE is_bad***</div>
<img align="middle" src=""></img-->

# END OF LECTURE

next lecture: [`neo4j`](012_dbs_4_neo4j.ipynb)